In [829]:
import simpy as smp
from random import expovariate, random, randint

In [830]:
def print_time(s, sec):
    mins = str(int(sec // 60))
    secs = int(sec %  60)
    if secs // 10 == 0:
        secs = "0" + str(secs)
    else:
        secs = str(secs)
    print(s, mins + ":" + secs)

In [831]:
def discrete_rv(drv):
    if sum([val[0] for val in drv]) != 1:
        raise 1
    r = random()
    s = 0
    for pair in drv:
        if (r > s) and (r < (s + pair[0])):
            return pair[1]
        s += pair[0]

In [832]:
CUSTOMER_DSTRB = [[0.5, 1], [0.3, 2], [0.1, 3], [0.1, 4]]
WAY_DSTRB = [[0.8, "hot"], [0.15, "cold"], [0.05, "drinks"]]
CASHBOX_NUMBER = 2
MEAN_GAP = 30
CUSTOMER_NUMBER = 0
STUDETNTS = []

In [833]:
def gen_group(env, canteen):
    global CUSTOMER_NUMBER, STUDETNTS
    env.process(canteen.start())
    while True:
        yield env.timeout(expovariate(1 / MEAN_GAP))
        n = discrete_rv(CUSTOMER_DSTRB)
        for i in range(n):
            std = Student(env, canteen, CUSTOMER_NUMBER)
            STUDETNTS += [std]
            env.process(std.start())
            CUSTOMER_NUMBER += 1

In [834]:
class Student:
    def __init__(self, env, canteen, index):
        self.env = env
        self.canteen = canteen
        self.index = index
        self.service_time = 0
        self.cashbox_time = 0
        self.delays = {"hot":0, "cold":0, "cash":0}
        self.cash_wait = 0
        self.finished = 0
    def start(self):
        self.service_time = self.env.now
        self.way = discrete_rv(WAY_DSTRB)
        print("Студент {0} направился к {1}".format(self.index, self.way))
        if self.way != "drinks":
            with self.canteen.stations[self.way][0].request() as req:
                self.canteen.stations[self.way][2] += 1
                wait = self.env.now
                yield req
                self.canteen.stations[self.way][2] -= 1
                self.delays[self.way] = self.env.now - wait
                print("Студент {0} начал обслуживание у {1}_station в {2}".format(self.index, self.way, self.env.now))
                yield self.env.process(self.canteen.stations[self.way][1](self))
                print("Студент {0} покинул {1}_station в {2}".format(self.index, self.way, self.env.now))
        
        print("Студент {0} начал обслуживание у drinks_station  в {1}".format(self.index, self.env.now))
        yield self.env.process(self.canteen.drinks_station(self))
        print("Студент {0} покинул drinks_station в {1}".format(self.index, self.env.now))
               
        cash_n, cashbox, q_len = min(self.canteen.cashboxes, key=lambda x: x[2])
        print([cbox[2] for cbox in self.canteen.cashboxes])
        print("Студент {0} прибыл к кассе #{1} в {2}".format(self.index, cash_n,  self.env.now))
        
        
        self.canteen.cashboxes[cash_n][2] += 1
        wait = self.env.now
        with cashbox.request() as cash_req:
            yield cash_req
            self.delays["cash"] = self.env.now - wait
            print("Студент {0} начал обслуживание у кассы #{1} в {2}".format(self.index, cash_n, self.env.now))
            yield self.env.process(self.canteen.cashbox(self))
            print("Студент {0} покинул кассу #{1} в {2}".format(self.index, cash_n, self.env.now))
            self.canteen.cashboxes[cash_n][2] -= 1
        self.finished = 1
        self.service_time = self.env.now - self.service_time
        print_time("Студент {0} обслуживался".format(self.index), self.service_time)

In [835]:
class Canteen:
    def __init__(self, env):
        self.env = env
        self.hot = smp.Resource(self.env, capacity=1)
        self.cold = smp.Resource(self.env, capacity=1)
        self.cashboxes = [[i, smp.Resource(self.env, capacity=1), 0] for i in range(CASHBOX_NUMBER)]
        self.stations = {"hot":[self.hot, self.hot_station, 0], "cold":[self.cold, self.cold_station, 0]}
    
    def hot_station(self, student):
        time = randint(50, 120)
        student.cashbox_time += randint(20, 40)
        print("hot time:", time)
        yield student.env.timeout(time)
    
    def cold_station(self, student):
        time = randint(60, 180)
        student.cashbox_time += randint(5, 15)
        print("{0} student's cold time:".format(student.index), time)
        yield student.env.timeout(time)
    
    def drinks_station(self, student):
        time = randint(5, 20)
        student.cashbox_time += randint(5, 10)
        print("drinks time:", time)
        yield student.env.timeout(time) 
    
    def cashbox(self, student):
        yield student.env.timeout(student.cashbox_time)
        
    def start(self):
        self.mean_hot_queue  = 0
        self.mean_cold_queue = 0
        self.mean_cash_queue = 0
        self.mean_all = 0
        
        self.max_hot_queue  = 0
        self.max_cold_queue = 0
        self.max_cash_queue = 0
        self.max_all = 0
        tick = 0
        while True:
            yield self.env.timeout(1)
            self.mean_hot_queue = self.mean_hot_queue * tick + self.stations["hot"][2]
            self.mean_cold_queue = self.mean_cold_queue * tick + self.stations["cold"][2]
            self.mean_cash_queue = self.mean_cash_queue * tick + sum([cash[2] for cash in self.cashboxes]) / CASHBOX_NUMBER
            all_students = self.stations["hot"][2] + self.stations["cold"][2] + sum([cash[2] for cash in self.cashboxes])
            self.mean_all = self.mean_all * tick + all_students
            tick += 1 
            
            self.mean_hot_queue /= tick
            self.mean_cold_queue /= tick
            self.mean_cash_queue /= tick
            self.mean_all /= tick
            
            if self.stations["hot"][2] > self.max_hot_queue:
                self.max_hot_queue = self.stations["hot"][2]
            if self.stations["cold"][2] > self.max_cold_queue:
                self.max_cold_queue = self.stations["cold"][2]
            if max([cash[2] for cash in self.cashboxes]) > self.max_cash_queue:
                self.max_cash_queue = max([cash[2] for cash in self.cashboxes])
            if all_students > self.max_all:
                self.max_all = all_students
        
        

In [836]:
env = smp.Environment()
cnt = Canteen(env)
env.process(gen_group(env, cnt))
env.run(until=90*60)

Студент 0 направился к cold
Студент 0 начал обслуживание у cold_station в 16.242097977983967
0 student's cold time: 151
Студент 1 направился к cold
Студент 2 направился к cold
Студент 3 направился к cold
Студент 4 направился к hot
Студент 5 направился к hot
Студент 4 начал обслуживание у hot_station в 129.1140138151285
hot time: 57
Студент 6 направился к hot
Студент 7 направился к hot
Студент 8 направился к hot
Студент 9 направился к hot
Студент 0 покинул cold_station в 167.24209797798397
Студент 0 начал обслуживание у drinks_station  в 167.24209797798397
drinks time: 9
Студент 1 начал обслуживание у cold_station в 167.24209797798397
1 student's cold time: 118
Студент 0 покинул drinks_station в 176.24209797798397
[0, 0]
Студент 0 прибыл к кассе #0 в 176.24209797798397
Студент 0 начал обслуживание у кассы #0 в 176.24209797798397
Студент 4 покинул hot_station в 186.1140138151285
Студент 4 начал обслуживание у drinks_station  в 186.1140138151285
drinks time: 11
Студент 5 начал обслуживани

In [837]:
mn_hot = sum([std.delays["hot"] for std in STUDETNTS if (std.way == "hot" and std.finished)]) / len([std for std in STUDETNTS if (std.way == "hot" and std.finished)])
print_time("Mean hot delay:", mn_hot)
mx_hot = max([std.delays["hot"] for std in STUDETNTS if std.way == "hot"])
print_time("Max  hot delay", mx_hot)

Mean hot delay: 31:20
Max  hot delay 64:48


In [838]:
mn_cold = sum([std.delays["cold"] for std in STUDETNTS if (std.way == "cold" and std.finished)]) / len([std for std in STUDETNTS if (std.way == "cold" and std.finished)])
print_time("Mean cold delay:", mn_cold)
mx_cold = max([std.delays["cold"] for std in STUDETNTS if std.way == "cold"])
print_time("Max  cold delay:", mx_cold)

Mean cold delay: 4:48
Max  cold delay: 11:49


In [839]:
mn_cash = sum([std.delays["cash"] for std in STUDETNTS if std.finished]) / len([std for std in STUDETNTS if std.finished])
mx_cash = max([std.delays["cash"] for std in STUDETNTS if std.finished])
print_time("Mean cash delay:", mn_cash)
print_time("Max  cash delay:", mx_cash)
max([std.delays["cash"] for std in STUDETNTS if std.finished])

Mean cash delay: 0:00
Max  cash delay: 0:16


16.8183546309574

In [840]:
print("Mean hot_station queue:", int(cnt.mean_hot_queue))
print("Max  hot_station queue:", cnt.max_hot_queue)

Mean hot_station queue: 108
Max  hot_station queue: 242


In [841]:
print("Mean cold_station queue:", int(cnt.mean_cold_queue))
print("Max  cold_station queue:", cnt.max_cold_queue)

Mean cold_station queue: 2
Max  cold_station queue: 8


In [842]:
mn_total = sum([sum(std.delays.values()) for std in STUDETNTS if std.finished]) / len([std for std in STUDETNTS if std.finished])
print_time("Mean total service time:", mn_total)

Mean total service time: 19:19


In [843]:
cnt.max_cash_queue

2

In [844]:
cnt.max_all

251

In [845]:
cnt.mean_all

111.19837006853099

In [846]:
print(len(STUDETNTS))
print(len([std for std in STUDETNTS if std.way == "hot"]))
print(len([std for std in STUDETNTS if std.way == "cold"]))
print(len([std for std in STUDETNTS if std.way not in ("hot", "cold")]))

366
308
46
12


In [847]:
308/366


0.8415300546448088